### Import Required Libraries and Set Up Environment Variables

In [99]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [100]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
query_url = f"{base_url}{CME}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

In [101]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(query_url)


In [103]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()


In [104]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[:1], indent=4))

[
    {
        "activityID": "2013-05-01T03:12:00-CME-001",
        "catalog": "M2M_CATALOG",
        "startTime": "2013-05-01T03:12Z",
        "instruments": [
            {
                "displayName": "SOHO: LASCO/C2"
            },
            {
                "displayName": "SOHO: LASCO/C3"
            },
            {
                "displayName": "STEREO A: SECCHI/COR2"
            },
            {
                "displayName": "STEREO B: SECCHI/COR2"
            }
        ],
        "sourceLocation": "",
        "activeRegionNum": null,
        "note": "",
        "submissionTime": "2013-08-07T16:54Z",
        "versionId": 1,
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
        "cmeAnalyses": [
            {
                "isMostAccurate": true,
                "time21_5": "2013-05-01T07:07Z",
                "latitude": 12.0,
                "longitude": -120.0,
                "halfAngle": 36.0,
                "speed": 860.0,
        

In [105]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)
# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[["activityID", "startTime", "linkedEvents"]]
cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [106]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df = cme_df.dropna(subset=['linkedEvents'])
cme_df.head()


,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [107]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for index, row in cme_df.iterrows():
  # Get the corresponding values from the current row
    activity_id = row['activityID']
    start_time = row['startTime']
    linked_events = row['linkedEvents']  # This contains the list of dictionaries   
    # Iterate over each dictionary in the list
    for event in linked_events:
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
        expanded_rows.append({
            'activityID': activity_id,
            'startTime': start_time,
            'linkedEvent': event
        })
# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)
expanded_df.head()

,activityID,startTime,linkedEvent
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}


In [111]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors

        # Log the error or print it for debugging

def extract_activityID_from_dict(input_dict):
    """
    Extracts the 'activityID' from a single dictionary within 'linkedEvents'.
    Handles exceptions to manage unexpected inputs gracefully.
    """
    try:
        # Ensure input is a dictionary
        if isinstance(input_dict, dict) and 'activityID' in input_dict:
            return input_dict['activityID']
        else:
            raise ValueError("Input is not a dictionary with 'activityID'.")
    except Exception as e:
        # Log the error for debugging purposes
        print(f"Error: {e}")
        return None  # Return None if an error occurs

# Example test with a single linkedEvent dictionary
example_linked_event = {'activityID': '2013-05-04T04:52:00-IPS-001'}

# Test the function
print(extract_activityID_from_dict(example_linked_event))





2013-05-04T04:52:00-IPS-001


In [122]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
# Create 'GST_ActivityID' column using the loc indexer
cme_df.loc[:, 'GST_ActivityID'] = cme_df['linkedEvents'].apply(
    lambda x: ", ".join(extract_activityID_from_list(x)) if extract_activityID_from_list(x) else None
)

# Display the updated DataFrame
cme_df[['activityID', 'startTime', 'linkedEvents', 'GST_ActivityID']].head()



,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}],2013-05-04T04:52:00-IPS-001
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}],2013-05-07T04:37:00-IPS-001
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}],2013-05-12T23:30:00-IPS-001
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...,"2013-05-13T01:53:00-FLR-001, 2013-05-13T04:12:..."
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...,"2013-05-13T15:40:00-FLR-001, 2013-05-13T18:02:..."


In [119]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
# Drop rows where 'GST_ActivityID' is missing or NaN
cme_df = cme_df.dropna(subset=['GST_ActivityID'])

# Display the updated DataFrame to verify the changes
cme_df[['activityID', 'startTime', 'GST_ActivityID']].head()


,activityID,startTime,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,2013-05-04T04:52:00-IPS-001
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,2013-05-07T04:37:00-IPS-001
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,2013-05-12T23:30:00-IPS-001
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,"2013-05-13T01:53:00-FLR-001, 2013-05-13T04:12:..."
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,"2013-05-13T15:40:00-FLR-001, 2013-05-13T18:02:..."


In [133]:
# print out the datatype of each column in this DataFrame:
# Print the data types of each column
cme_df.info()



<class 'pandas.core.frame.DataFrame'>
Index: 1023 entries, 0 to 5523
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           1023 non-null   object             
 1   startTime_CME   1023 non-null   datetime64[ns, UTC]
 2   GST_ActivityID  1023 non-null   object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 32.0+ KB


In [135]:
# Ensure 'GST_ActivityID' exists and convert to string format
if 'GST_ActivityID' in cme_df.columns:
    cme_df['GST_ActivityID'] = cme_df['GST_ActivityID'].fillna('').astype(str)
else:
    print("Warning: 'GST_ActivityID' column does not exist!")

# Ensure 'startTime' exists and convert to datetime format
if 'startTime' in cme_df.columns:
    cme_df['startTime'] = pd.to_datetime(cme_df['startTime'], errors='coerce', utc=True)
else:
    print("Error: 'startTime' column does not exist!")

# Rename columns if they exist
rename_mapping = {'startTime': 'startTime_CME', 'activityID': 'cmeID'}
cme_df.rename(columns=rename_mapping, inplace=True)

# Drop 'linkedEvents' column if it exists
if 'linkedEvents' in cme_df.columns:
    cme_df.drop(columns=['linkedEvents'], inplace=True)

# Verify the final DataFrame
print(cme_df.info())
print(cme_df.head())






Error: 'startTime' column does not exist!
<class 'pandas.core.frame.DataFrame'>
Index: 1023 entries, 0 to 5523
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           1023 non-null   object             
 1   startTime_CME   1023 non-null   datetime64[ns, UTC]
 2   GST_ActivityID  1023 non-null   object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 32.0+ KB
None
                          cmeID             startTime_CME  \
0   2013-05-01T03:12:00-CME-001 2013-05-01 03:12:00+00:00   
4   2013-05-03T22:36:00-CME-001 2013-05-03 22:36:00+00:00   
7   2013-05-09T19:29:00-CME-001 2013-05-09 19:29:00+00:00   
10  2013-05-13T02:54:00-CME-001 2013-05-13 02:54:00+00:00   
13  2013-05-13T16:18:00-CME-001 2013-05-13 16:18:00+00:00   

                                       GST_ActivityID  
0                         2013-05-04T04:52:00-IPS-001  
4                      

In [136]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
# Keep only rows where the 'GST_ActivityID' column contains 'GST'
cme_df = cme_df[cme_df['GST_ActivityID'].str.contains('GST', na=False)]

# Verify the filtered DataFrame
cme_df[['cmeID', 'startTime_CME', 'GST_ActivityID']].head()


,cmeID,startTime_CME,GST_ActivityID
26,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
137,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,"2013-09-30T05:05:00-SEP-001, 2013-10-02T01:54:..."
238,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,"2013-12-08T00:00:00-IPS-001, 2013-12-08T00:00:..."
398,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,"2014-02-18T05:00:00-IPS-001, 2014-02-19T03:00:..."
402,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,"2014-02-20T02:50:00-IPS-001, 2014-02-20T03:00:..."


### GST Data

In [137]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build the URL for GST data
gst_url = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

# Print the constructed URL to verify
print("Constructed GST URL:", gst_url)

Constructed GST URL: https://api.nasa.gov/DONKI/GST?startDate=2013-05-01&endDate=2024-05-01&api_key=fo5N4netF2QUPuKYwHozZZT42PDtPfaiskuSbhXc


In [138]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_url)

In [139]:


# Use json.dumps with argument indent=4 to format data


# Convert the response to JSON and store it in a variable
gst_json = gst_response.json()

# Preview the first result in JSON format
print("First Result (Formatted):")
print(json.dumps(gst_json[0], indent=4))  # Format the first result with indentation


First Result (Formatted):
{
    "gstID": "2013-06-01T01:00:00-GST-001",
    "startTime": "2013-06-01T01:00Z",
    "allKpIndex": [
        {
            "observedTime": "2013-06-01T01:00Z",
            "kpIndex": 6.0,
            "source": "NOAA"
        }
    ],
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
    "linkedEvents": [
        {
            "activityID": "2013-05-31T15:45:00-HSS-001"
        }
    ],
    "submissionTime": "2013-07-15T19:26Z",
    "versionId": 1
}


In [144]:
# Convert gst_json to a Pandas DataFrame
gst_df = pd.DataFrame(gst_json)

# Keep only the specified columns: 'activityID', 'startTime', 'linkedEvents'
gst_df = gst_df[['gstID', 'startTime', 'linkedEvents']]

# Verify the result
gst_df.head()


,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...


In [145]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows where 'linkedEvents' is missing or empty
gst_df = gst_df[gst_df['linkedEvents'].notnull() & gst_df['linkedEvents'].str.len() > 0]

# Verify the result
gst_df.head()  # Display the first few rows of the updated DataFrame


,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [146]:
# Ensure 'linkedEvents' is not null and is iterable before exploding
gst_df = gst_df[gst_df['linkedEvents'].notnull()]

# Explode the 'linkedEvents' column so each element becomes its own row
gst_df = gst_df.explode('linkedEvents').reset_index(drop=True)

# Drop rows where 'linkedEvents' is still missing or null after exploding
gst_df = gst_df.dropna(subset=['linkedEvents'])

# Verify the result
gst_df.head()  # Preview the first few rows


,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'}
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'}
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'}
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'}
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'}


In [147]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
def extract_activityID_from_dict(event_dict):
    try:
        # Check if the dictionary contains 'activityID' with 'CME'
        if isinstance(event_dict, dict) and 'activityID' in event_dict and 'CME' in event_dict['activityID']:
            return event_dict['activityID']
        return None  # Return None if no 'CME' activityID is found
    except TypeError:
        return None  # Handle cases where event_dict is not a dictionary

# Apply the function to the 'linkedEvents' column and create a new column 'CME_ActivityID'
gst_df.loc[:, 'CME_ActivityID'] = gst_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

# Remove rows with missing 'CME_ActivityID'
gst_df = gst_df[gst_df['CME_ActivityID'].notnull()]

# Verify the results

gst_df.head()  # Preview the updated DataFrame


,gstID,startTime,linkedEvents,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
5,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,{'activityID': '2013-12-04T23:12:00-CME-001'},2013-12-04T23:12:00-CME-001
7,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,{'activityID': '2014-02-16T14:15:00-CME-001'},2014-02-16T14:15:00-CME-001
9,2014-02-20T03:00:00-GST-001,2014-02-20T03:00Z,{'activityID': '2014-02-18T01:25:00-CME-001'},2014-02-18T01:25:00-CME-001


In [148]:
# Convert the 'CME_ActivityID' column to string format
gst_df['CME_ActivityID'] = gst_df['CME_ActivityID'].astype(str)

# Convert the 'gstID' column to string format
gst_df['gstID'] = gst_df['gstID'].astype(str)

# Convert 'startTime' to datetime format
gst_df['startTime'] = pd.to_datetime(gst_df['startTime'], errors='coerce', utc=True)

# Rename 'startTime' to 'startTime_GST'
gst_df.rename(columns={'startTime': 'startTime_GST'}, inplace=True)

# Drop the 'linkedEvents' column
gst_df.drop(columns=['linkedEvents'], inplace=True)

# Verify all steps were executed correctly
print(gst_df.info())  # Check the structure and data types
print(gst_df.head())  # Preview the updated DataFrame


<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 1 to 203
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           61 non-null     object             
 1   startTime_GST   61 non-null     datetime64[ns, UTC]
 2   CME_ActivityID  61 non-null     object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 1.9+ KB
None
                         gstID             startTime_GST  \
1  2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
2  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
5  2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00   
7  2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00   
9  2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00   

                CME_ActivityID  
1  2013-06-02T20:24:00-CME-001  
2  2013-09-29T22:40:00-CME-001  
5  2013-12-04T23:12:00-CME-001  
7  2014-02-16T14:15:00-CME-001  
9  2014-02-18T01:25:00

In [150]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
# Filter rows where 'CME_ActivityID' contains 'CME'
gst_df = gst_df[gst_df['CME_ActivityID'].str.contains('CME', na=False)]

# Verify the result by checking the first few rows
print(gst_df.info())  # Check the structure and data types of the filtered DataFrame
print(gst_df.head())  # Preview the first few rows of the filtered DataFrame


<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 1 to 203
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           61 non-null     object             
 1   startTime_GST   61 non-null     datetime64[ns, UTC]
 2   CME_ActivityID  61 non-null     object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 1.9+ KB
None
                         gstID             startTime_GST  \
1  2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
2  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
5  2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00   
7  2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00   
9  2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00   

                CME_ActivityID  
1  2013-06-02T20:24:00-CME-001  
2  2013-09-29T22:40:00-CME-001  
5  2013-12-04T23:12:00-CME-001  
7  2014-02-16T14:15:00-CME-001  
9  2014-02-18T01:25:00

### Merge both datatsets

In [163]:
# Merge the two DataFrames
merged_df = gst_df.merge(
    cme_df,
    left_on=['gstID', 'CME_ActivityID'],  # Columns in gst_df
    right_on=['GST_ActivityID', 'cmeID'],  # Columns in cme_df
    how='inner'  # Use an inner join to keep only matching rows
)

# Verify the merged DataFrame
merged_df.head()  # Preview the first few rows of the merged DataFrame



,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
1,2015-09-09T03:00:00-GST-001,2015-09-09 03:00:00+00:00,2015-09-04T14:12:00-CME-001,2015-09-04T14:12:00-CME-001,2015-09-04 14:12:00+00:00,2015-09-09T03:00:00-GST-001
2,2015-09-09T03:00:00-GST-001,2015-09-09 03:00:00+00:00,2015-09-04T19:24:00-CME-001,2015-09-04T19:24:00-CME-001,2015-09-04 19:24:00+00:00,2015-09-09T03:00:00-GST-001
3,2015-09-11T06:00:00-GST-001,2015-09-11 06:00:00+00:00,2015-09-06T23:30:00-CME-001,2015-09-06T23:30:00-CME-001,2015-09-06 23:30:00+00:00,2015-09-11T06:00:00-GST-001
4,2015-09-11T06:00:00-GST-001,2015-09-11 06:00:00+00:00,2015-09-07T10:36:00-CME-001,2015-09-07T10:36:00-CME-001,2015-09-07 10:36:00+00:00,2015-09-11T06:00:00-GST-001


In [175]:
# Verify that the new DataFrame has the same number of rows as cme and gst
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           5 non-null      object             
 1   startTime_GST   5 non-null      datetime64[ns, UTC]
 2   CME_ActivityID  5 non-null      object             
 3   cmeID           5 non-null      object             
 4   startTime_CME   5 non-null      datetime64[ns, UTC]
 5   GST_ActivityID  5 non-null      object             
 6   timeDiff        5 non-null      timedelta64[ns]    
dtypes: datetime64[ns, UTC](2), object(4), timedelta64[ns](1)
memory usage: 408.0+ bytes


### Computing the time it takes for a CME to cause a GST

In [174]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
# Ensure 'startTime_GST' and 'startTime_CME' are in datetime format
merged_df['startTime_GST'] = pd.to_datetime(merged_df['startTime_GST'], errors='coerce', utc=True)
merged_df['startTime_CME'] = pd.to_datetime(merged_df['startTime_CME'], errors='coerce', utc=True)

# Compute the time difference between 'startTime_GST' and 'startTime_CME'
merged_df['timeDiff'] = merged_df['startTime_GST'] - merged_df['startTime_CME']

# Verify the new column
merged_df[['startTime_GST', 'startTime_CME', 'timeDiff']].head()  # Preview time differences


,startTime_GST,startTime_CME,timeDiff
0,2013-06-07 03:00:00+00:00,2013-06-02 20:24:00+00:00,4 days 06:36:00
1,2015-09-09 03:00:00+00:00,2015-09-04 14:12:00+00:00,4 days 12:48:00
2,2015-09-09 03:00:00+00:00,2015-09-04 19:24:00+00:00,4 days 07:36:00
3,2015-09-11 06:00:00+00:00,2015-09-06 23:30:00+00:00,4 days 06:30:00
4,2015-09-11 06:00:00+00:00,2015-09-07 10:36:00+00:00,3 days 19:24:00


In [173]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
# Ensure the 'timeDiff' column is in timedelta format (it should already be in this format from previous steps)
# Compute mean and median time in days
mean_time = merged_df['timeDiff'].mean()
median_time = merged_df['timeDiff'].median()

# Display the descriptive statistics for 'timeDiff'
print(merged_df['timeDiff'].describe())  # Provides additional insights like min, max, std
print(f"\nMean time difference: {mean_time}")
print(f"Median time difference: {median_time}")


count                            5
mean               4 days 05:46:48
std      0 days 06:21:16.859924386
min                3 days 19:24:00
25%                4 days 06:30:00
50%                4 days 06:36:00
75%                4 days 07:36:00
max                4 days 12:48:00
Name: timeDiff, dtype: object

Mean time difference: 4 days 05:46:48
Median time difference: 4 days 06:36:00


### Exporting data in csv format

In [170]:
# Export data to CSV without the index
# Export the DataFrame to a CSV file without the index
output_file = "merged_cme_gst_data.csv"
merged_df.to_csv(output_file, index=False)

print(f"Data exported successfully to {output_file}")


Data exported successfully to merged_cme_gst_data.csv
